In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_boston


In [ ]:
data = load_boston()
data_pd = pd.DataFrame(data.data, columns=data.feature_names)
data_pd['price'] = data.target
data_pd.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [ ]:
#标准化X, y
y = np.array(data_pd['price'])
y = (y - np.mean(y)) / np.sqrt(np.mean((y-np.mean(y))**2))
y = y.reshape(506,1)
d = data_pd
d = np.array(d.drop(['price'], axis=1))
dt = d.T
X = ((dt - np.mean(dt, axis=1).reshape(13,1)) / np.sqrt(np.mean((dt - np.mean(dt, axis=1).reshape(13,1))**2, axis=1)).reshape(13,1)).T



In [ ]:
#proximal算法

def proximal(A, b, lamda, miu, epsilon):   #lamda:正则项 miu:步长
  x_dim = A.shape[1]
  beta_old = np.zeros(x_dim).reshape(x_dim, 1)
  beta_temp = beta_old - np.dot(miu * A.T, np.dot(A, beta_old) - b)
  beta_new = (np.maximum(1 - lamda*miu/abs(beta_temp), 0) * beta_temp)
  k = 1
  while np.sum(abs(beta_new - beta_old))/np.sum(abs(beta_old)) >= epsilon:
    beta_old = beta_new
    beta_temp = beta_old - np.dot(miu * A.T, np.dot(A, beta_old) - b)
    beta_new = (np.maximum(1 - lamda*miu/abs(beta_temp), 0) * beta_temp)
    k += 1
    #迭代次数超过1百万次时退出
    if (k >= 1e6):
      break 
  return beta_new

def proximal2(A, b, lamda, miu, epsilon):   #lamda:正则项 miu:步长
  plot_record = []
  x_dim = A.shape[1]
  beta_old = np.zeros(x_dim).reshape(x_dim, 1)
  plot_record.append(beta_old)
  beta_temp = beta_old - np.dot(miu * A.T, np.dot(A, beta_old) - b)
  beta_new = (np.maximum(1 - lamda*miu/abs(beta_temp), 0) * beta_temp)
  plot_record.append(beta_new)
  k = 1
  while np.sum(abs(beta_new - beta_old))/np.sum(abs(beta_old)) >= epsilon:
    beta_old = beta_new
    beta_temp = beta_old - np.dot(miu * A.T, np.dot(A, beta_old) - b)
    beta_new = (np.maximum(1 - lamda*miu/abs(beta_temp), 0) * beta_temp)
    plot_record.append(beta_new)
    k += 1
    #迭代次数超过1百万次时退出
    if (k >= 1e6):
      break 
  return beta_new, plot_record


In [ ]:
#交叉验证
num_k = 5
miu = 4e-6
epsilon = 1e-10
def get_kfold_data(data, k):
  fold_data=[]
  fold_size = data.shape[0] // k
  for i in range(k):
    if i != k-1:
      f_start = i * fold_size
      f_end = (i+1) * fold_size
      fold_data.append(data[f_start:f_end])
    else:
      f_start = i * fold_size
      fold_data.append(data[f_start:])
  return fold_data

X_folds = get_kfold_data(X,num_k)
y_folds = get_kfold_data(y,num_k)

lamda_to_rsqure={}
l=np.linspace(0,60,61)
lamda_choices = 10**(-3+0.1*l)

for lamda in lamda_choices:
  rsq=[]
  for i in range(num_k):
    X_train = np.concatenate([fold for j, fold in enumerate(X_folds) if j != i])
    y_train = np.concatenate([fold for j, fold in enumerate(y_folds) if j != i])
    X_val = X_folds[i]
    y_val = y_folds[i]
    beta_h = proximal(X_train, y_train, lamda, miu, epsilon)
    y_pred = np.dot(X_val, beta_h)
    rsq.append(1 - float(np.sum((y_pred - y_val)**2)/np.sum((y_val - np.mean(y_val))**2)))
  lamda_to_rsqure[lamda] = np.mean(rsq)

for lamda in sorted(lamda_to_rsqure):
    print("lambda = %f, Rsqure = %f"%(lamda, lamda_to_rsqure[lamda]))



  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


lambda = 0.001000, Rsqure = 0.435284
lambda = 0.001259, Rsqure = 0.435286
lambda = 0.001585, Rsqure = 0.435288
lambda = 0.001995, Rsqure = 0.435291
lambda = 0.002512, Rsqure = 0.435295
lambda = 0.003162, Rsqure = 0.435300
lambda = 0.003981, Rsqure = 0.435306
lambda = 0.005012, Rsqure = 0.435313
lambda = 0.006310, Rsqure = 0.435322
lambda = 0.007943, Rsqure = 0.435334
lambda = 0.010000, Rsqure = 0.435349
lambda = 0.012589, Rsqure = 0.435367
lambda = 0.015849, Rsqure = 0.435391
lambda = 0.019953, Rsqure = 0.435420
lambda = 0.025119, Rsqure = 0.435457
lambda = 0.031623, Rsqure = 0.435503
lambda = 0.039811, Rsqure = 0.435561
lambda = 0.050119, Rsqure = 0.435634
lambda = 0.063096, Rsqure = 0.435726
lambda = 0.079433, Rsqure = 0.435841
lambda = 0.100000, Rsqure = 0.435984
lambda = 0.125893, Rsqure = 0.436164
lambda = 0.158489, Rsqure = 0.436387
lambda = 0.199526, Rsqure = 0.436665
lambda = 0.251189, Rsqure = 0.437008
lambda = 0.316228, Rsqure = 0.437432
lambda = 0.398107, Rsqure = 0.437951
l

In [ ]:
best_lamda = lamda_choices[np.array([i for _,i in lamda_to_rsqure.items()]).argmax()]
# best_lamda 5.011872336272725
beta_final, plot_beta = proximal2(X, y, best_lamda, miu, epsilon)
beta_final

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in double_scalars


array([[-0.07178775],
       [ 0.08063836],
       [-0.        ],
       [ 0.07188364],
       [-0.17565605],
       [ 0.30604116],
       [-0.        ],
       [-0.2706715 ],
       [ 0.14414257],
       [-0.10337248],
       [-0.21047756],
       [ 0.08378861],
       [-0.40555378]])

In [ ]:
predict_y = np.dot(X, beta_final)
1 - float(np.sum((predict_y - y)**2)/np.sum((y - np.mean(y))**2))

0.7362233400724643

In [ ]:
# len(plot_beta) 81336
# plot_b = np.array(plot_beta).reshape(81336,13).T
for i in range(len(plot_b)):
   plt.plot(np.linspace(0,81336,81336), plot_b[i], label=data.feature_names[i])

plt.legend()